In [ ]:
!pip install geopy

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
df = pd.read_csv('/content/ff_race.csv', sep=";")
df.head()

,Place,First,Last,City,State,Age,Division,Time
0,1,Daniel,Wilson,Tulsa,OK,35,M,8:23:01
1,2,Eric,Davis,Greenwood,IN,38,M,8:57:54
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,9:24:35
3,4,Ron,Hammett,Montverde,FL,53,M,9:24:36
4,5,Seth,Cain,Geneva,FL,44,M,9:42:17


Перевірка структури та дані

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Place     109 non-null    int64 
 1   First     109 non-null    object
 2   Last      109 non-null    object
 3   City      109 non-null    object
 4   State     109 non-null    object
 5   Age       109 non-null    int64 
 6   Division  109 non-null    object
 7   Time      109 non-null    object
dtypes: int64(2), object(6)
memory usage: 6.9+ KB


In [ ]:
df.columns

Index(['Place', 'First', 'Last', 'City', 'State', 'Age', 'Division', 'Time'], dtype='object')

Видалення дублікатів

In [ ]:
df = df.drop_duplicates()

Перетворення age на число

In [ ]:
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

Перевірка на пропущені значення

In [ ]:
df.isna().sum()

,0
Place,0
First,0
Last,0
City,0
State,0
Age,0
Division,0
Time,0


In [ ]:
df.dtypes

,0
Place,int64
First,object
Last,object
City,object
State,object
Age,int64
Division,object
Time,object


Обробка часу

In [ ]:
df['Time'] = pd.to_timedelta(df['Time'], errors='coerce')

In [ ]:
df['Time_minutes'] = df['Time'].dt.total_seconds()/60

In [ ]:
# Оставляем только строки с временем меньше 1 суток
df = df[df["Time"] < pd.Timedelta(days=1)]

In [ ]:
# Преобразуем timedelta в строку HH:MM:SS
df["Time"] = df["Time"].apply(lambda x: str(x).split(" ")[-1] if pd.notna(x) else "")

In [ ]:
df.head()

,Place,First,Last,City,State,Age,Division,Time,Time_minutes
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01,503.016667
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54,537.900000
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35,564.583333
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36,564.600000
4,5,Seth,Cain,Geneva,FL,44,M,09:42:17,582.283333


In [ ]:
df['Time_minutes'] = df['Time_minutes'].round().astype(int)
df.head()

,Place,First,Last,City,State,Age,Division,Time,Time_minutes
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01,503
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54,538
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35,565
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36,565
4,5,Seth,Cain,Geneva,FL,44,M,09:42:17,582


Об`єднання імен

In [ ]:
df['Name'] = df['First'] + ' ' + df['Last']

In [ ]:
df.head()

,Place,First,Last,City,State,Age,Division,Time,Time_minutes,Name
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01,503,Daniel Wilson
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54,538,Eric Davis
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35,565,Stewart Edwards
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36,565,Ron Hammett
4,5,Seth,Cain,Geneva,FL,44,M,09:42:17,582,Seth Cain


In [ ]:
df.rename(columns={'Division': 'Gender'}, inplace=True)

In [ ]:
df.head()

,Place,First,Last,City,State,Age,Gender,Time,Time_minutes,Name
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01,503,Daniel Wilson
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54,538,Eric Davis
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35,565,Stewart Edwards
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36,565,Ron Hammett
4,5,Seth,Cain,Geneva,FL,44,M,09:42:17,582,Seth Cain


In [ ]:
df.to_csv('ff_race_2.0.csv', index=False)